# Black-Litterman Model

$$
\begin{align}
\max_{\mathbf{w}} \quad & \mathbf{w}^T \bar{\mu} - \frac{\delta}{2} \mathbf{w}^T \hat{\Sigma} \mathbf{w}  &\\
\text{s.t.: } \quad & \boldsymbol 1^T \mathbf{w}^T = 1 \\
\quad & \mathbf{w} \ge 0

\end{align}
$$

where $\bar{\mu}$ is defined as 

$$
\bar{\mu} = [ (\tau \Sigma)^{-1} + P \Omega^{-1} P]^{-1} [(\tau \Sigma)^{-1} \Pi + P \Omega^{-1} Q]
$$

and

$$
\bar{M}^{-1} = [(\tau \Sigma)^{-1} + P \Omega^{-1} P]^{-1}
$$

and $\hat{\Sigma} = \Sigma + \bar{M}^{-1}$

where:
- $\Pi$ is the market equilibrium expected return vector, defined as follows: $\Pi = \delta \Sigma \mathbf{w}_{\textit{mkt}}$.
- $\mathbf{P}$ is the $K \times N$ matrix that maps the investor's views to the asset returns. Each row corresponds to a view, and the columns indicate the assets.
- $\mathbf{Q}$ is a $K$-vector of the expected return on the portfolios defined by $\mathbf{P}$.
- $\Omega$ is a $K \times K$ the diagonal matrix representing the uncertainty (variances) associated with the views.
- $\tau$ is the scaling factor that determines the strength of the views.

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from port_opt import get_optimal_portfolio, neg_utility_value

In [12]:
close_df = pd.read_csv('..\data\DK\preprocessed_data\danish_closed_stocks.csv', index_col=0)
mu = np.load('..\data\DK\preprocessed_data\stocks\m.npy')
cov = np.load('..\data\DK\preprocessed_data\stocks\S.npy')

stocks_names = close_df.columns.tolist()

In [4]:
mkt_caps = {stock: 0 for stock in stocks_names}

mkt_caps['MAERSK'] = 216.976 # biliion DKK
mkt_caps['ORSTED'] = 222.972 # biliion DKK
mkt_caps['VWS'] =  157.775 # biliion DKK
mkt_caps['CHR'] =  59.581 # biliion DKK
mkt_caps['NOVO'] =  2.783e3 # trilion DKK
mkt_caps['NZYM'] =  82.782 # bilion DKK
mkt_caps['ZEAL'] =  15.196 # bilion DKK
mkt_caps['DNORD'] =  10.964 # bilion DKK
mkt_caps['TRMD'] = 15.225 # bilion DKK
mkt_caps['STG'] = 9.521 # bilion DKK
mkt_caps['SOLAR'] =  3.432 # bilion DKK
mkt_caps['AOJ'] =  2.074 # bilion DKK
mkt_caps['SKAKO'] =  236.088e-3 # milion DKK
mkt_caps['NDA'] = 256.213 # bilion DKK
mkt_caps['LUXOR'] =  540.375e-3 # milion DKK
mkt_caps['PNDORA'] =  58.699 # bilion DKK

In [6]:
cap_sum = sum(list(mkt_caps.values()))
cap_weights = {k: v / cap_sum for k, v in mkt_caps.items()}

In [14]:
w_mkt = np.array(list(cap_weights.values()))

The process that I've undertaken for constructing the matrices $\mathbf{P}$, $\mathbf{Q}$, and $\mathbf{\Omega}$ for the Black-Litterman model was the following:

1. $\mathbf{P}$: To construct $\mathbf{P}$, I analyzed historical data to identify how different sectors perform in various economic cycles. I created four scenarios to represent each part of the economic cycle: Early, Mid, Late, and Recession. Each scenario corresponds to a row in $\mathbf{P}$, and the stocks' expected returns for each scenario are based on the historical performance of their respective sectors during similar economic cycles. So, each row in $\mathbf{P}$ captures the expected returns of stocks aligned with their sector's performance in a specific economic scenario.

2. $\mathbf{Q}$: My vector $\mathbf{Q}$ contains probabilities that represent the likelihood of each economic scenario occurring in the upcoming quarters. I assign higher probabilities to scenarios that I believe are more likely to happen based on my analysis. This helps me express my expectations about the potential economic conditions.

3. $\mathbf{\Omega}$: For the matrix $\mathbf{\Omega}$, I account for the uncertainty associated with each view. I've considered that views with a lower likelihood of occurring in the near future are less certain, so I assign higher values to those. Conversely, for scenarios that I believe are more likely, I assign lower values to indicate higher confidence in those views.

The shortcoming of this approach is that the sector returns that I've used are not even close to be realistic, and they are the same for each stock beloning to the same sector. The ideal approach would be to sample from a mutlinormal distribution with the expected returns for each sector and for each part of the economic cycle, with an appropriate covariance matrix.

In [29]:
tau = 0.1
delta = 0.2
P = np.array([
    [0., 0., 0.2, 0.1, 0.2, 0.1, -0.2, -0.2, -0.2, 0.2, 0.2, 0.2, 0., 0.1, 0.2, -0.2, -0.2],
    [-0.10, -0.10, 0., 0., 0., 0., 0., -0.1, 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0.10, 0.10, 0., 0., 0., 0., 0., 0., 0.1, -0.2, 0., 0., 0.1, 0., 0., 0.2, 0.],
    [0.20, 0.20, -0.2, -0.2, -0.2, -0.2, 0.2, 0.2, 0.2, 0.1, -0.2, -0.2, 0.2, -0.2, -0.2, 0., 0.2]
])
Q = np.array([0.1, 0.15, 0.3, 0.45])
Omg = np.diag([0.25, 0.2, 0.15, 0.1])
Pi = delta * (cov @ w_mkt)

In [18]:
for stock in stocks_names:
    print(stock, end=' ')

AOJ CHR DNORD LUXOR MAERSK NDA NOVO NZYM ORSTED PNDORA SKAKO SOLAR STG TOP TRMD VWS ZEAL 

```python
sector_dict = {
    'Consumer Discretionary': ['PNDORA'],
    'Consumer Staples': ['CHR', 'STG', 'AOJ'],
    'Finance': ['NDA', 'LUXOR'],
    'Energy': ['ORSTED'],
    'Industrial': ['SOLAR', 'MAERKS', 'DNORD', 'TRMD', 'SKAKO'],
    'Health Care': ['NOVO', 'NZYM', 'ZEAL'],
    'Energy': ['VWS'],
    'Utilities': ['ORSTED']
}

```

In [30]:
M_1 = np.linalg.inv(np.linalg.inv(tau * cov) + P.T @ np.linalg.inv(Omg) @ P)

In [31]:
mu_bar = M_1 @ (np.linalg.inv(tau * cov) @ Pi + P.T @ np.linalg.inv(Omg) @ Q)
Sigma_bar = cov + M_1

In [32]:
# w_star = 1 / delta * np.linalg.inv(Sigma_bar) @ M_1 @ (np.linalg.inv(tau * cov) @ Pi + P.T @ np.linalg.inv(Omg) @ Q)

In [35]:
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
res = get_optimal_portfolio(
    f_obj=neg_utility_value,
    args=(mu_bar, Sigma_bar, delta),
    constraints=constraints,
    bounds=tuple((0,1) for _ in range(len(stocks_names))))

In [42]:
res

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.0032370310425407743
       x: [ 2.104e-19  1.737e-01 ...  6.389e-18  2.452e-01]
     nit: 15
     jac: [ 3.710e-03 -2.832e-03 ...  2.766e-03 -3.230e-03]
    nfev: 270
    njev: 15

In [44]:
print(f'Return: {100*res.fun:.2f}%\n')

for stock, weight in zip(stocks_names, res.x):
     print(f'{stock}: {100*weight:.2f}% ', end="", flush=True)

Return: -0.32%

AOJ: 0.00% CHR: 17.37% DNORD: 0.00% LUXOR: 0.00% MAERSK: 0.00% NDA: 0.00% NOVO: 37.13% NZYM: 0.00% ORSTED: 20.98% PNDORA: 0.00% SKAKO: 0.00% SOLAR: 0.00% STG: 0.00% TOP: 0.00% TRMD: 0.00% VWS: 0.00% ZEAL: 24.52% 

# Todos
- [x] Market cap of the Danish stocks